In [2]:
import os
import sys
import pickle
import json
import numpy as np
import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
sys.path.append(os.path.dirname(os.getcwd()))

device = torch.device("cpu")

In [10]:
with open("../asset/Kolmogorov_evolution.pkl", "rb") as file:
    df, states_vorticity = pickle.load(file)

mpl.rcdefaults()
mpl.style.use("../configs/mplrc")
mpl.rc("figure.subplot", wspace=0.25)
mpl.rc("axes.spines", bottom=False, left=False)

titles = ["Low-resolution\nobservations", "Grid sparse\nobservations", "Partial\nobservations"]

for title, fname, data in zip(titles, ["AvgPool", "GridMask", "CenterMask"], df):
    observations_vorticity = data["observations_vorticity"]
    mean_vorticity = data["mean_vorticity"]
    nrows = 1
    ncols = 3
    fig, axes = plt.subplots(
        nrows=nrows,
        ncols=ncols, 
        figsize=(6, 3),
    )

    vmin_state, vmax_state = -0.6, 0.6
    state_norm = mpl.colors.Normalize(vmin=vmin_state, vmax=vmax_state)

    for ax in axes:
        ax.xaxis.set_visible(False)
        ax.yaxis.set_ticks([])

    ims = []
    for i in range(51):
        im_ref = axes[0].imshow(states_vorticity[i], cmap=sns.cm.icefire, norm=state_norm)
        axes[0].set_title("Ref. state")
        im_obs = axes[1].imshow(observations_vorticity[i], cmap=sns.cm.icefire)
        axes[1].set_title(title)
        im_ssls = axes[2].imshow(mean_vorticity[i], cmap=sns.cm.icefire, norm=state_norm)
        axes[2].set_title("SSLS")
        ims.append((im_ref, im_obs, im_ssls))

    ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True, repeat_delay=1000)
    writer = animation.PillowWriter(fps=10, bitrate=1800)
    ani.save(f"../asset/{fname}.gif", writer=writer, dpi=100)
    # writer = animation.FFMpegWriter(fps=10, bitrate=1800)
    # ani.save(f"../asset/{fname}.mp4", writer=writer)

    plt.close()